# Range Mask Example

In [1]:
import numpy as np
import xarray as xr
import echopype.mask

**Create a mock xarray dataset to exemplify filters that mask data based on depth range.**

In [2]:
def get_mock_range_mask_data(n: int, n_chan_freq: int) -> xr.Dataset:
    """
    Creates an in-memory mock Sv Dataset.

    Parameters
    ----------
    n: int
        The number of rows (``ping_time``) and columns (``range_sample``) of
        each channel matrix
    n_chan_freq: int
        Determines the size of the ``channel`` coordinate

    Returns
    -------
    mock_Sv_ds: xr.Dataset
        A mock Sv dataset to be used for ``mask_range`` tests. The Sv
        data values for the channel coordinate ``chan1`` will be equal to ``mat_A``,
        ``chan3`` will be equal to ``mat_B``, and all other channel coordinates
        will retain the value of ``np.identity(n)``.

    """

    if n_chan_freq < 1:
        raise RuntimeError("The input n_chan_freq must be greater than or equal to 1!")

    # matrix representing freqB
    mat_B = np.arange(n ** 2).reshape(n, n) - np.identity(n)

    # matrix representing freqA
    mat_A = np.arange(n ** 2).reshape(n, n)

    # construct channel values
    chan_vals = ['chan' + str(i) for i in range(1, n_chan_freq + 1)]

    # construct mock Sv data
    mock_Sv_data = [mat_A, np.identity(n), mat_B] + [np.identity(n) for i in range(3, n_chan_freq)]

    # set channel coordinate name
    channel_coord_name = "channel"

    # create mock Sv DataArray
    mock_Sv_da = xr.DataArray(data=np.stack(mock_Sv_data),
                              coords={channel_coord_name: chan_vals, "ping_time": np.arange(n),
                                      "range_sample": np.arange(n)})

    # create data variables for the Dataset
    data_vars = {"Sv": mock_Sv_da}


    # create mock Dataset with Sv and frequency_nominal
    mock_Sv_ds = xr.Dataset(data_vars=data_vars)

    return mock_Sv_ds

Sv_ds = get_mock_range_mask_data(6, 3)

In [3]:
Sv_ds

<xarray.Dataset>
Dimensions:       (channel: 3, ping_time: 6, range_sample: 6)
Coordinates:
  * channel       (channel) <U5 'chan1' 'chan2' 'chan3'
  * ping_time     (ping_time) int64 0 1 2 3 4 5
  * range_sample  (range_sample) int64 0 1 2 3 4 5
Data variables:
    Sv            (channel, ping_time, range_sample) float64 0.0 1.0 ... 34.0

**Specific masking techniques  examples**
- Techniques: ‘Above’, ‘Below’, ‘Inside’, ‘Outside’

In [4]:
print("\033[1m CHANNEL 1:\033[0m\n")
print(f"\033[1m Sv Test Data:\033[0m\n {Sv_ds.sel(channel='chan1').Sv.values}")
print(f"\033[1m Range Test Data:\033[0m\n {Sv_ds.range_sample.values}")
print(f"\033[1m Below 1m mask:\033[0m\n" 
      f"{echopype.mask.get_range_mask(Sv_ds, channel='chan1', r0=1, method ='below')}")
print(f"\033[1m Above 1m mask:\033[0m\n" 
      f"{echopype.mask.get_range_mask(Sv_ds, channel='chan1', r0=1, method ='above')}")
print(f"\033[1m Inside 1m-3m mask:\033[0m\n" 
      f"{echopype.mask.get_range_mask(Sv_ds, channel='chan1', r0=1, r1=3, method ='inside')}")
print(f"\033[1m Outside 1m-3m mask:\033[0m\n" 
      f"{echopype.mask.get_range_mask(Sv_ds, channel='chan1', r0=1, r1=3, method ='outside')}")

 CHANNEL 1:

 Sv Test Data:
 [[ 0.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34. 35.]]
 Range Test Data:
 [0 1 2 3 4 5]
 Below 1m mask:
<xarray.DataArray (ping_time: 6, range_sample: 6)>
array([[False, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True]])
Coordinates:
  * ping_time     (ping_time) int64 0 1 2 3 4 5
  * range_sample  (range_sample) int64 0 1 2 3 4 5
 Above 1m mask:
<xarray.DataArray (ping_time: 6, range_sample: 6)>
array([[ True, False, False, False, False, False],
       [ True, False, False, False, False, False],
       [ True, False, False, False, False, False],
       [ True, False, False, False, False, False],
       [ True, Fal